# Lowered & typed code

## An introduction to code transformations in Julia

Tim Holy

JuliaCon 2021

# Multiple ways to write the same code

```julia
if isempty(list)
    idx = nothing
else
    idx = firstindex(list)
end
```

is the same as

```julia
idx = isempty(list) ? nothing : firstindex(list)
```


```julia
function f(x, y)
    return x + y
end
```

is the same as

```julia
f(x, y) = x + y
```

Other examples: `do`-block syntax, `for` vs `while` loops, etc.

# Some higher-level constructs add complexity

```julia
function myfunc(a, b)
    f(z) = z^2 + a      # an inner function
    
    ...
end
```

Scoping rules make the naive parsing of code less straightforward:

In [ ]:
function f()
    for i = 1:5
        x = rand()
    end
    println("The last random number was ", x)
end

f()

Expressing these with uniform syntax is the main purpose of *lowering*.

# @code_lowered

In [ ]:
f(a, b) = a*a + b
@code_lowered f(1, 2)

%n: [Single Static Assignment](https://en.wikipedia.org/wiki/Static_single_assignment_form) value

Left column: [basic block](https://en.wikipedia.org/wiki/Basic_block)

In [ ]:
g(cond, val1, val2) = cond ? val1 + val2 : val2
@code_lowered g(true, 1, 2)

In [ ]:
function blastoff()
    counter = 5
    while counter > 0
        println(counter)
        counter -= 1
    end
    println("blast off!")
end
@code_lowered blastoff()

In [ ]:
function mysum(list)
    ss = 0.0
    for item in list
        ss += item
    end
    return ss
end
@code_lowered mysum([1,2,3])

In [ ]:
# We can see scoping in action:
function f()
    for i = 1:5
        x = rand()
    end
    println("The last random number was ", x)
end
@code_lowered f()

# Lowered representation of more complex constructs

An example using inner functions:

In [ ]:
function myfunc(a, b)
    f(z) = z^2 + a      # an inner function

    return f(b)
end
@code_lowered myfunc(1, 1)

What is `#f#1`?

Conceptually, Julia expands it like this:

```julia
struct FClosure{A}
    a::A
end
(f::FClosure)(z) = z^2 + f.a

function myfunc(a, b)
    fc = FClosure{typeof(a)}(a)
    return fc(b)
end
```

`#f#1` is the automatically-generated name given to `FClosure`.

We can see this with `Meta.lower(mod, expr)`:

In [ ]:
Meta.lower(Main, quote
    function myfunc(a, b)
        f(z) = z^2 + a      # an inner function

        return f(b)
    end
end)

# @code_typed

This allows you to see the results of type-inference:

In [ ]:
f(a, b) = a*a + b
@code_lowered f(1, 2)

In [ ]:
@code_typed optimize=false f(1, 2)

In [ ]:
@code_typed optimize=true f(1, 2)

In [ ]:
@code_typed f(1.0, 2)

# Summary

- Each `Method` has a single lowered-code representation
- See it with `@code_lowered` or, in complex cases, `Meta.lower`
- Lowering expands macros, resolves and flattens scope, and reduces control-flow to "goto"s
- `@code_typed` also shows type inference (corresponding to a particular `MethodInstance`)
- `@code_typed` by default applies optimization transformations like inlining and linearization

With practice, these code representations become quite readable